In [1]:
%matplotlib inline

import os,sys

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
from torch.autograd import Variable

import timeit

%load_ext autoreload
%autoreload 2

In [2]:
def apply_mirror_boundary_conditions(coord, dim):
    """
    Return the correct coordinate according to mirror boundary conditions
        coord: a coordinate (x or y) in the image
        dim: the length of the axis of said coordinate
    """
    # If the coordinate is outside of the bounds of the axis, take its reflection inside the image
    if coord < 0:
        coord = -coord
    elif coord >= dim:
        coord =  2*(dim-1) - coord % (2*(dim-1))
    # Else, do nothing
    return int(coord)

def get_window(image, window_size, centre_coordinates):
    """
    Get a window in image taking into account boundary conditions
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
        centre_coordinates: a list containing the x-y coordinates of the window's central pixel
    """
    # Get convenient variables
    window_radius = (window_size - 1)/2
    i_centre, j_centre = (centre_coordinates[0], centre_coordinates[1])
    nrows, ncols, nchannels = image.shape
    window = np.zeros((window_size, window_size, nchannels))
    # Fill in the window array with pixels of the image
    for i in range(window_size):
        # Apply mirror boundary conditions on the x-coordinate
        i_mirrored = apply_mirror_boundary_conditions(i_centre + i - window_radius, nrows)
        for j in range(window_size):
            # Same for the y-coordinate
            j_mirrored = apply_mirror_boundary_conditions(j_centre + j - window_radius, ncols)
            # Fill in the window with the corresponding pixel
            window[i, j, :] = image[i_mirrored, j_mirrored, :]
    return window

In [3]:
for j in range(4, 3, -1):
    print(j)

4


In [4]:
def shift_to_the_right(image, window, centre_coordinates, step=1):
    nrows, ncols, _ = image.shape
    window_size = len(window)
    window_radius = (window_size - 1)/2
    #j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + step + window_radius, ncols)
    shifted = np.roll(window, -step, axis=1)
    for i in range(window_size):
        i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + i - window_radius, nrows)            
        for j in range(window_size-step, window_size):
            j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + j - window_radius + step, ncols)
            shifted[i, j, :] = image[i_mirrored, j_mirrored, :]
    return shifted

def shift_to_the_bottom(image, window, centre_coordinates, step=1):
    nrows, ncols, _ = image.shape
    window_size = len(window)
    window_radius = (window_size - 1)/2
    #i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + 1 + window_radius, nrows)
    shifted = np.roll(window, -step, axis=0)
    #print('shifted\n', shifted[:,:,0], '\n')
    for j in range(window_size):
        j_mirrored = apply_mirror_boundary_conditions(centre_coordinates[1] + j - window_radius, ncols)
        for i in range(window_size-step, window_size):
            i_mirrored = apply_mirror_boundary_conditions(centre_coordinates[0] + i - window_radius + step, nrows)
            #print(i, i_mirrored)
            shifted[i, j, :] = image[i_mirrored, j_mirrored, :]
    return shifted

def sliding_window(image, window_size, step=1):
    """
    Construct a list of sliding windows of given size on an image.
    The windows will slide from left to right and from up to down.
        image: a numpy array representing our image
        window_size: an odd number specifying the size of the window
        step: the value of the shift between windows
    """
    nrows, ncols, _ = image.shape
    windows = []
    i = 0
    row_windows = [get_window(image, window_size, [i, 0])]
    for j in range(0, ncols-1, step):
        row_windows += [shift_to_the_right(image, row_windows[-1], [i, j], step)]
    windows += row_windows
    for i in range(0, nrows-1, step):
        row_windows = [shift_to_the_bottom(image, row_windows[int(j/step)], [i, j], step) for j in range(0, ncols, step)]
        windows += row_windows
    return windows

In [5]:
# Directory and files name
train_dir = "training/"
tr_image_dir = train_dir + "images/"
tr_label_dir = train_dir + "groundtruth/"

tr_image_files = os.listdir(tr_image_dir)
tr_label_files = os.listdir(tr_label_dir)

# Number of training samples
N = len(tr_image_files)

# Load the images and ground truth
img_train = []
label_train = []
for i in range(N):
    img = mpimg.imread(tr_image_dir + tr_image_files[i])
    label = mpimg.imread(tr_label_dir + tr_label_files[i])
    
    img_train.append(img)
    label_train.append(label)

# Keep only sub-set of images
NUM_IMAGES = N

img_train = np.asarray(img_train[:NUM_IMAGES])
label_train = np.asarray(label_train[:NUM_IMAGES])

print(img_train.shape, label_train.shape)

(100, 400, 400, 3) (100, 400, 400)


In [6]:
#im = img_train[0]
#for window_size in [3, 5, 7, 15, 35, 51]:
#    %timeit sliding_window(im, window_size, step=int(window_size/2))

In [7]:
def compute_all_windows(img_train, label_train, window_size, step):
    train_data = []
    train_labels = []
    for im, labels, i in zip(img_train, label_train, range(len(img_train))):
        w_im = sliding_window(im, window_size, step)
        w_labels = sliding_window(labels[:, :, np.newaxis], window_size, step)
        train_data += w_im
        train_labels += w_labels
        path = './windows_train/' + str(i)
        os.makedirs(path, exist_ok=True)
        for wi, wl, j in zip(w_im, w_labels, range(len(w_im))):
            img_name = path + '/im_' + str(j) + '.png'
            plt.imsave(img_name, wi)
            label_name = path + '/label_' + str(j) + '.png'
            plt.imsave(label_name, wl[:,:,0])
    return train_data, train_labels

In [8]:
window_size = 51
step = int(window_size/2)
train_data, train_labels = compute_all_windows(img_train, label_train, window_size, step)

In [9]:
len(train_data) == len(train_labels)

True

In [10]:
train_d = np.array(train_data)
train_l = np.array(train_labels)[:, :, :, 0]

In [15]:
train_d = train_d.reshape((len(train_data)*51, 51, 3))
train_l = train_l.reshape((len(train_labels)*51, 51))

In [10]:
#a = np.array(range(10*10))
#a = a.reshape((10, 10))
#print(a)
#aa = get_window(a[:,:,np.newaxis], 5, [5,5])[:,:,0]
#print('\n')
#print(aa)
#print('\n')
#print(shift_to_the_bottom(a[:,:,np.newaxis], aa[:,:,np.newaxis], [5,5], 2)[:,:,0])

In [16]:
NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
IMG_PATCH_SIZE = 51
TRAINING_SIZE = 27300
BATCH_SIZE = 32
NUM_EPOCHS = 15
a = 0.00

In [17]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, LeakyReLU, Input, Reshape, Permute, Average
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Lambda, Activation, GlobalAveragePooling2D, Conv2DTranspose, GlobalAveragePooling1D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [18]:
model = Sequential()
model.add(Conv2D(32, 2, input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(32, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(LeakyReLU(alpha=a))
model.add(Dense(1, activation='sigmoid'))

In [19]:
from keras.engine.topology import Layer
def fire(x, squeeze, expand):
    x = Conv2D(squeeze, 1)(x)
    x = LeakyReLU(alpha=a)(x)
    e11 = Conv2D(expand, 1)(x)
    e11 = LeakyReLU(alpha=a)(e11)
    e33 = Conv2D(expand, 1)(x)
    e33 = LeakyReLU(alpha=a)(e33)
    return Concatenate(axis=3)([e11, e33])

In [20]:
inputs = Input(shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS))
x = Conv2D(32, kernel_size=3, strides=2, input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS))(inputs)
x = LeakyReLU(alpha=a)(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

x = fire(x, squeeze=8, expand=16)
x = fire(x, squeeze=8, expand=16)
x = MaxPooling2D(pool_size=3, strides=2)(x)

'''x = fire(x, squeeze=32, expand=128)
x = fire(x, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=3, strides=2)(x)

x = fire(x, squeeze=48, expand=192)
x = fire(x, squeeze=48, expand=192)
x = fire(x, squeeze=64, expand=256)
x = fire(x, squeeze=64, expand=256)
x = MaxPooling2D(pool_size=3, strides=2)(x)'''
x = Flatten()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs, x)

In [21]:
model = Sequential()
model.add(Conv2D(32, 2, input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(32, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(Conv2D(64, 2))
model.add(LeakyReLU(alpha=a))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(2, 1))
model.add(Conv2DTranspose(1, kernel_size=IMG_PATCH_SIZE - 1, padding='valid'))
#model.add(Reshape((-1, IMG_PATCH_SIZE * IMG_PATCH_SIZE)))
#model.add(Permute((2,1)))
model.add(GlobalAveragePooling2D())
model.add(Activation('sigmoid'))

In [22]:
for layer in model.layers:
    print(layer.input_shape,layer.output_shape )

(None, 51, 51, 3) (None, 50, 50, 32)
(None, 50, 50, 32) (None, 50, 50, 32)
(None, 50, 50, 32) (None, 49, 49, 32)
(None, 49, 49, 32) (None, 49, 49, 32)
(None, 49, 49, 32) (None, 24, 24, 32)
(None, 24, 24, 32) (None, 24, 24, 32)
(None, 24, 24, 32) (None, 23, 23, 64)
(None, 23, 23, 64) (None, 23, 23, 64)
(None, 23, 23, 64) (None, 22, 22, 64)
(None, 22, 22, 64) (None, 22, 22, 64)
(None, 22, 22, 64) (None, 11, 11, 64)
(None, 11, 11, 64) (None, 11, 11, 64)
(None, 11, 11, 64) (None, 11, 11, 2)
(None, 11, 11, 2) (None, 60, 60, 1)
(None, 60, 60, 1) (None, 1)
(None, 1) (None, 1)


In [23]:
LR = 0.001
DECAY = 0.00000
adam = Adam(lr=LR, decay=DECAY)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

model.fit(train_d, train_l, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)

ValueError: Error when checking input: expected conv2d_12_input to have 4 dimensions, but got array with shape (1392300, 51, 3)

In [ ]:
score = model.evaluate(train_data, train_labels)
score